# Analysis of Capital, Fixed, and Variable costs for Generation Types

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Many of these reports are from previous years, so we should account for this by adjusting for inflation. 
We can do this with the ``cpi`` (consumer price index) from the Bureau of Labor Statistics.

$$ P_{adj} = \frac{P_{old} \cdot CPI_{present}}{CPI_{old}} $$

We can retreive the consumer price index data from the Bureau of Labor Statistics. The usual values for inflation calculations are the "All Urban Consumers (CU)" survey. The code for this is "CUUR0000SA0". 

In [2]:
url = "https://download.bls.gov/pub/time.series/cu/cu.data.0.Current"
cpi_data = pd.read_csv(url, sep='\t')

In [3]:
cpi_key = cpi_data.keys()[0]
cpi_key

'series_id        '

In [4]:
cu = list(cpi_data[cpi_key])[0]
cu

'CUSR0000SA0      '

In [5]:
cpi_u = cpi_data[cpi_key] == cu
cpi_data = cpi_data[cpi_u]

In [6]:
cpi_data.index = pd.to_datetime(cpi_data.iloc[:,1], format='%Y')
cpi_data

,series_id,year,period,value,footnote_codes
year,,,,,
1997-01-01,CUSR0000SA0,1997,M01,159.400,NaN
1997-01-01,CUSR0000SA0,1997,M02,159.700,NaN
1997-01-01,CUSR0000SA0,1997,M03,159.800,NaN
1997-01-01,CUSR0000SA0,1997,M04,159.900,NaN
1997-01-01,CUSR0000SA0,1997,M05,159.900,NaN
...,...,...,...,...,...
2020-01-01,CUSR0000SA0,2020,M10,260.462,NaN
2020-01-01,CUSR0000SA0,2020,M11,260.927,NaN
2020-01-01,CUSR0000SA0,2020,M12,261.560,NaN


In [7]:
cpi_average = cpi_data.resample('Y').mean()
cpi_average

,year,value,footnote_codes
year,,,
1997-12-31,1997,160.525000,NaN
1998-12-31,1998,163.008333,NaN
1999-12-31,1999,166.583333,NaN
2000-12-31,2000,172.191667,NaN
2001-12-31,2001,177.041667,NaN
2002-12-31,2002,179.866667,NaN
2003-12-31,2003,184.000000,NaN
2004-12-31,2004,188.908333,NaN
2005-12-31,2005,195.266667,NaN


In [8]:
target_year = 2021
cpi_current = cpi_average[cpi_average.iloc[:,0]==target_year].iloc[:,1]

In [9]:
def inflate(p_old, start_year, target_year=2021):
    """
    This function calculates the inflated value of an item
    based on the Consumer Price Index from the Bureau of 
    Labor Statistics.
    
    Parameters
    ----------
    p_old : float
        The old price or value of an item. I.e. the price of
        the item at the starting year.
    start_year : integer
        The year from which you would like to inflate. E.g.
        the price of milk in 1997 inflated to today's dollars.
    target_year : integer
        The target year you would like to inflate to. E.g.
        the price of milk in 1997 inflated to 2021 dollars.
        Default is 2021.
        
    Returns
    -------
    p_adj : float
        The price adjusted for inflation.
    """
    
    cpi_current = float(cpi_average[cpi_average.iloc[:,0]==target_year].iloc[:,1])    
    cpi_old = float(cpi_average[cpi_average.iloc[:,0]==start_year].iloc[:,1])
    
    p_adj = (p_old*cpi_current)/cpi_old
    
    return p_adj

In [10]:
inflate(1.00, 2017, 2017)

1.0

In [11]:
inflate(1.00, 2017, 2020)

1.0559224728092558

In [12]:
inflate(1.00, 2017, 1997)

0.6548419139618701

## Nuclear Plants

"Nuclear Costs in Context" (NEI, 2018)

This document gives a cost summary for each of fuel, capital, and operating costs in \$/MWh. This is 
helpful for variable operating costs. But we cannot reliably convert these into fixed values required for capital and fixed annual costs. 

In [13]:
fuel_nei_bwr = inflate(6.22, 2017)/1e6  # M$/MWh
fuel_nei_pwr = inflate(6.55, 2017)/1e6  # M$/MWh
print(fuel_nei_bwr);print(fuel_nei_pwr)

6.665575243079848e-06
7.019215087166078e-06


In [19]:
om_nei_bwr_total = inflate(27.81, 2017)/1e6  # M$/MWh
om_nei_pwr_total = inflate(26.36, 2017)/1e6  # M$/MWh
print(om_nei_bwr_total);print(om_nei_pwr_total)

2.9802194133448643e-05
2.8248322091251573e-05


"Projected Costs of Generating Electricity" (IEA, 2020)

This document only gives a cost summary for overnight capital costs in \$/kWe. This is helpful for investment costs but gives no indication for annual fixed costs. It also does not distinguish between PWR and BWR for the United States.

It DOES give values for capital costs required for license renewals (if we were to incorporate this, I believe we should implement as an additional annual fixed cost, dividing by the life of the extension). 

In [15]:
# New Build for Nuclear
cc_iea = inflate(4250, 2020)*1000/1e6  # M$/MW
print(cc_iea)

4.313245200054472


"Nuclear Power Economics and Project Structuring" (WNA, 2017)

This document gives OM and Fuel Costs (cents per kWh) as well as estimated capital cost -- assumes PWR

In [16]:
om_total_wna = inflate(0.24, 2011)*1000/1e6  # M$/MWh
cc_wna_pwr = inflate(4100, 2015)*1000/1e6  # M$/MW
print(om_total_wna);print(cc_wna_pwr)

0.0002803049932643616
4.54449640139788


"Capital Costs and Performance Characteristics for Utility Scale Power Generating Technologies" (Sargent and Lundy, 2020)


This document gives capital cost, dollars/kW, variable OM, dollars/MWh, and fixed OM, \$ /kW-year. However, not for current technology in the U.S. Estimates for "advanced" (AP-1000) and "modular" (small modular reactors, 50 MW).

Also provided: Specific emissions information (NOx, SOx, CO2)

In [20]:
cc_sl_adv = inflate(6041, 2020)*1000/1e6  # M$/MW
cc_sl_smr = inflate(6191, 2020)*1000/1e6  # M$/MW
print(cc_sl_adv);print(cc_sl_smr)
om_sl_adv_fix = inflate(121.64, 2020)*1000/1e6  # M$/MW-year
om_sl_smr_fix = inflate(95.00, 2020)*1000/1e6  # M$/MW-year
print(om_sl_adv_fix);print(om_sl_smr_fix)
om_sl_adv_var = inflate(2.37, 2020)/1e6  # M$/MWh
om_sl_smr_var = inflate(3.00, 2020)/1e6  # M$/MWh
print(om_sl_adv_var);print(om_sl_smr_var)

6.1308974714186055
6.28312965494994
0.12345015203167674
0.09641371623651175
2.405268499795083e-06
3.044643670626687e-06


Based on the values from the Sargent and Lundy report, we can make a reasonable guess about the 
annual fixed cost of a BWR or PWR from the NEI-2018 report.

Assumptions:

1. The "capital" cost in \$/MWh corresponds to the annual "fixed" cost.
2. The "total operating" cost in \$/MWh corresponds to the "variable" cost.
3. The average capacity factor is 92\%. This will be used to convert MWh to MW

In [22]:
mwh_to_mw = 8760*0.92
om_nei_bwr_fix = inflate(6.63*mwh_to_mw, 2017)/1e6  # M$/MW-year
om_nei_pwr_fix = inflate(6.64*mwh_to_mw, 2017)/1e6  # M$/MW-year
print(om_nei_bwr_fix);print(om_nei_pwr_fix)

0.057260180468418506
0.05734654574815971


In [23]:
nuclear_pd = {'capital':[np.nan,np.nan,
                         cc_iea,
                         cc_wna_pwr,
                         cc_sl_adv, cc_sl_smr,
                         ],  # M$/MW
              'fixed':[om_nei_pwr_fix,om_nei_bwr_fix,
                       np.nan,
                       np.nan,
                       om_sl_adv_fix, om_sl_smr_fix,
                       ],  # M$/MW-year
              'variable':[om_nei_pwr_total, om_nei_bwr_total,
                          np.nan,
                          om_total_wna,
                          om_sl_adv_var, om_sl_smr_var],  # M$/MWh
              'type':['PWR', 'BWR',
                      'LWR',
                      'PWR',
                      'advanced', 'modular'],  # BWR, PWR, advanced, modular
              'source':['NEI-2018','NEI-2018',
                        'IEA-2020',
                        'WNA-2017',
                        'SL-2020', 'SL-2020'],
              'notes':['calculated from assumptions tabulated in cost_analysis.ipynb',
                       'calculated from assumptions tabulated in cost_analysis.ipynb',
                       'source does not give annual fixed or variable costs',
                       'source does not give annual fixed cost',
                       'source specifies \"advanced\" as AP-1000',
                       'source specifies \"modular\" as 12x50-MW capacity']
             }  

nuclear_df = pd.DataFrame(nuclear_pd)
nuclear_df

,capital,fixed,variable,type,source,notes
0,NaN,0.057347,0.000028,PWR,NEI-2018,calculated from assumptions tabulated in cost_...
1,NaN,0.057260,0.000030,BWR,NEI-2018,calculated from assumptions tabulated in cost_...
2,4.313245,NaN,NaN,LWR,IEA-2020,source does not give annual fixed or variable ...
3,4.544496,NaN,0.000280,PWR,WNA-2017,source does not give annual fixed cost
4,6.130897,0.123450,0.000002,advanced,SL-2020,"source specifies ""advanced"" as AP-1000"
5,6.283130,0.096414,0.000003,modular,SL-2020,"source specifies ""modular"" as 12x50-MW capacity"
